In [1]:
import torch
import matplotlib.pyplot as plt
import numpy as np 
import argparse
import pickle 
import os
from PIL import Image
import json
from os import listdir, path

In [2]:
from pycocotools.coco import COCO

In [3]:
from caption import caption_image_beam_search, caption_image_beam_search1, visualize_att

In [4]:
coco = COCO('./data/annotations/captions_flickr30k.json')

loading annotations into memory...
Done (t=0.05s)
creating index...
index created!


In [5]:
img_ids = set()
for k in coco.anns.keys():
    img_ids.add(coco.anns[k]['image_id'])

In [6]:
images = []
for img_id in img_ids:
    images.append('data/Flickr30k/flickr30k-images/'+coco.loadImgs(img_id)[0]['file_name'])

In [7]:
import warnings
warnings.filterwarnings('ignore')
device = torch.device('cpu')

args = {
    'model':'./models/BEST_checkpoint_coco_5_cap_per_img_5_min_word_freq.pth.tar',
    'word_map':'./models/WORDMAP_coco_5_cap_per_img_5_min_word_freq.json'
}
img_path = "./img/"    #### provide path where image is stored

resultsFile = input("Name of output file: ")
store = open('./results/' + resultsFile,'w')

filename = listdir(img_path)
filenames = [f for f in filename if '.jpg' in f or '.png' in f or '.jpeg' in f]



# Load model
checkpoint = torch.load(args['model'], map_location=torch.device('cpu'))
decoder = checkpoint['decoder']
decoder = decoder.to(device)
decoder.eval()
encoder = checkpoint['encoder']
encoder = encoder.to(device)
encoder.eval()

# Load word map (word2idx)
with open(args['word_map'], 'r') as j:
    word_map = json.load(j)

# idx2word
rev_word_map = {v: k for k, v in word_map.items()}


for i, filename in enumerate(filenames):
    try:
        complete_seqs, complete_seqs_alpha = caption_image_beam_search1(encoder, decoder, img_path+filename, word_map, beam_size=5)
        captions = ""
        for j in range(len(complete_seqs)):
            seq = complete_seqs[j]
            sampled_caption = []
            for ind in seq[1:]:
                word = rev_word_map[ind]
                if word == '<end>':
                    break
                sampled_caption.append(word)
            sentence = ' '.join(sampled_caption)
            captions = captions + sentence + " "
        captions = captions.replace(".", " ")
        captions = captions.replace("   ", " ").replace("  ", " ").lower()
        # Print out the image and the generated caption
        filename = filename.split(".")[0]
        while filename[0] == "0":
            filename = filename[1:]
        print(filename+":%s" % (captions))
        store.write(filename+":" + captions + "\n")
    except KeyboardInterrupt:
        store.close()
        break
store.close()

9:a lunch box filled with different types of food a lunch box filled with a variety of foods a lunch box filled with different types of foods a lunch box with a variety of food items a lunch box with a variety of foods in it 
25:a tall giraffe standing next to a tree a giraffe standing next to a tall tree a giraffe standing in the middle of a forest a giraffe standing next to a tree in a forest a tall giraffe standing next to a tree in a forest 
30:a white vase filled with lots of flowers a white vase with a bouquet of flowers a white vase filled with red and white flowers a white vase filled with lots of different colored flowers a close up of a vase with flowers in it 
34:a zebra eating grass in a field a zebra grazing in a grassy field a zebra grazing on grass in a field a zebra grazing in a field of grass a zebra is grazing in a grassy field 
36:a woman in a dress holding a pink umbrella a woman in a bikini holding a pink umbrella a woman holding a pink umbrella on a beach a woman 